# Coronavirus progression dashboard - Overview
- Objective is to build a tool to track the progression of the Covid-19 pandemic, with up to date and reliable data
- Data comes for the Johs Hopkins University, the source usually used in media outlets (e.g. ABC news, Le Monde...)

The reaon for this dashboard to exist is that data usually available is either:
- incomplete (e.g. limitied number of coutries, limited time period)
- poorly presented (e.g. different  scales for diffrent coutries, lack of context)
- not granular enough (e.g. no breakdown by state for the US or Australia)

This dashboard uses all data available from Johns Hopkins and offers the following functionalities:
- Easy to use: the output is an .html file than can be sinky opened in a web browser. No installation of any sort is required\
- Easy to share: the .html file can simply be sent by email
- Free: No licence is required to ue and manipulate the dashboard
- Easy to update with new date: Update process takes ~15s
- Comprehensive: all time series avaiable from John Hopkins is included
- Flexible: The user can decide what to display and  how to doplay it

The dashboard is a very flexible tool and the user can play with the following:
List of avaialble metrics: 
- Number of cumulated Confirmed cases, by day or week
- Number of cumulated Deaths, by day or week
- Number of Confirmed cases, by day or week
- Number of Deaths, by day or week
- 7-days Rolling averages of Confirmed cases, by day or week
- 7-days Rolling averages of Deaths, by day or week

Data can be displayed by Coutry, Statem, or filtered on  a selection of countries or states. For instance, it is possible to display the evolution of the total number of cases by State in the US, or by Country in Europe, or the totals worlwide.
Data can also be filterd by date (e.g. display only the lates date to see the total number currently confirmed cases)

User can also chose how to visualize the data:
- Line chart of the 7-day rolling average of confirmed cases to see the trend of a selection of coutries
- Bar chart of or the death rate by country 
- Many more visualizations are avaible (table, heatmaps, area charts....)

# Import and display options

In [5]:
import pandas as pd
import numpy as np
from pivottablejs import pivot_ui
# Pandas options
pd.options.display.max_rows = 999
pd.set_option('expand_frame_repr', False)
# jupyter options
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline

# Define functions
- string_to_date turn a string column to datetime in a Panadas DF
- read_and_stack read the global data and stack it for furher manipulations
- read_and_stack_US read the US data and stack it for furher manipulations

In [6]:
def _string_to_date(s, format):
    """
    This is an extremely fast approach to datetime parsing.
    For large data, the same dates are often repeated. Rather than
    re-parse these, we store all unique dates, parse them, and
    use a lookup to convert all dates.
    """
    dates = {date:pd.to_datetime(date, format = format) for date in s.unique()}
    return s.map(dates)

def _read_and_stack(file):
    """ Reads the data and format it in long format with .stack() 
    with one row by country, province and date.
    This format is more flexible than the original format and can be
    use to quickly build a dashboard with pivottablejs

    Parameters
    ----------
    file: String
        the file to import, from the COVID-19 repository on github:
        https://github.com/CSSEGISandData/COVID-19.git
        
    Returns
    -------
    - df:
        pandas Series, stacked, with 1 row per ['Province/State', 'Country/Region', 'Date', 'Week']
    
    See Also
    --------

    Examples
    --------
    confirmed = _read_and_stack("Confirmed")

    """
    
    df = (pd
          .read_csv(rf"csse_covid_19_data\csse_covid_19_time_series\time_series_covid19_{file}_global.csv")
          .reset_index()
          .drop(["index", "Lat", "Long"], axis=1)
          .set_index(['Province/State', 'Country/Region'])
          .stack()
          .reset_index()
         )
    df["Date"] = _string_to_date(df["level_2"], "%m/%d/%y")
    df["Week"] = df.Date.dt.year * 100 + df.Date.dt.week
    df = (df
          .set_index(['Province/State', 'Country/Region', 'Date', 'Week'])
          .drop("level_2", axis=1)
         )
    df.columns = [file]
    
    return df[file]


def _read_and_stack_US(file):
    """ Reads the data and format it in long format with .stack() 
    with one row by country, province, county and date.
    This format is more flexible than the original format and can be
    use to quickly build a dashboard with pivottablejs
    
    Parameters
    ----------
    file: String
        the file to import, from the COVID-19 repository on github:
        https://github.com/CSSEGISandData/COVID-19.git
        
    Returns
    -------
    - df:
        pandas Series, stacked, with 1 row per ['Province/State', 'Country/Region', 'Date', 'Week']
    
    See Also
    --------

    Examples
    --------
    confirmed = _read_and_stack("Confirmed")

    """
    
    df = (pd
          .read_csv(rf"csse_covid_19_data\csse_covid_19_time_series\time_series_covid19_{file}_US.csv")
          .reset_index()
          .rename(columns={ "Province_State":"Province/State"
                          , "Country_Region": "Country/Region"
                          , "Admin2":"County"}
                 )
          .drop([  "index", "Lat", "Long", "Long_"
                 , "UID", "iso2", "iso3", "code3"
                 , "FIPS", "Combined_Key", "Population"]
                , axis=1, errors="ignore")
          .set_index(['Province/State', 'Country/Region', 'County'])
          .stack()
          .reset_index()
         )
    df["Date"] = _string_to_date(df["level_3"], "%m/%d/%y")
    df["Week"] = df.Date.dt.year * 100 + df.Date.dt.week
    df = (df
          .set_index(['Province/State', 'Country/Region', "County", 'Date', 'Week'])
          .drop("level_3", axis=1)
         )
    df.columns = [file]
    
    return df[file]



# df = _read_and_stack("Confirmed")
# # df.columns
# df.head()

# Covid-19 progression dashboard
Per country, province and date: confirmed, death, new confirmed and new deaths

## Prepare the data

In [7]:
# 1. Build a sigle DataFrame with count of Confirmed cases and count of Deaths 
# by country, province and date
# 1.1 Global data
files=["Confirmed", "Deaths"]
df_global = pd.DataFrame()
for file in files:
    df_global[file] = _read_and_stack(file)

df_global = df_global.reset_index()

# 1.2 US data
# a. Read raw data - county level
df_US = pd.DataFrame()
for file in files:
    df_US[file] = _read_and_stack_US(file)

df_US = df_US.reset_index()

# b. aggregate at State level to be consistent with the global data
df_US = (df_US
         .groupby(['Country/Region', 'Province/State', 'Date', 'Week'])
         .sum()
         .reset_index()
        )


# 2. Clean up the data
# 2.1 Compile Global and US data
df = pd.concat([  df_global[df_global["Country/Region"] != "US"]
                , df_US]
                , sort=True
              )
df =  df[["Country/Region", "Province/State", "Date", "Week", "Confirmed", "Deaths"]]


# 2.2 Replace missing Province/State by the name of the country
""" When Province/State is mussing , it refers to the main bulk of 
the country. e.g for France, when Province/State is missing the data refers 
to metropolitan France, as oppose to some specific Province
"""
df['Province/State'] = df['Province/State'].fillna(df['Country/Region'])

# 2.3 sort by Country, Province and date
df = (df
      .sort_values(['Country/Region', "Province/State", "Date"])
      .reset_index()
      .drop("index", axis=1)
     )

# 3. Add the mortality rate, number of new confirmed and number of new deaths
"""
rate = death / confirmed
new_confirmed = daily count of confirmed cases
new_deaths = dailty count of deaths
"""

# 3 Add the mortality rate, number of new confirmed and number of new deaths
df["rate"] = (df.Deaths / df.Confirmed).fillna(0)
df["New Confirmed"] = df.groupby(['Country/Region', "Province/State"]).Confirmed.diff().fillna(0)
df["New Deaths"] = df.groupby(['Country/Region', "Province/State"]).Deaths.diff().fillna(0)

# 4. Add rolling averages for New cases and new deaths
rolling = df.groupby(['Country/Region', "Province/State"])["New Confirmed", "New Deaths"].rolling(7).mean()
rolling.columns = ["New Confirmed Rolling", "New Deaths Rolling"]
# df = df.set_index(['Country/Region', "Province/State"])
df["New Confirmed Rolling"] = rolling.reset_index()["New Confirmed Rolling"]
df["New Deaths Rolling"] = rolling.reset_index()["New Deaths Rolling"]

# df.loc[df["Country/Region"] == "US"]

' When Province/State is mussing , it refers to the main bulk of \nthe country. e.g for France, when Province/State is missing the data refers \nto metropolitan France, as oppose to some specific Province\n'

'\nrate = death / confirmed\nnew_confirmed = daily count of confirmed cases\nnew_deaths = dailty count of deaths\n'

## Export the dashboard with pivottablejs.pivot_ui

In [8]:
# Export the interactive table / chart
pivot_ui(df,outfile_path="time_series_pivotchart.html")

# Rescaled dashboard to compare progression per country

## Rescale to log scale and start when the number of confirmed cases reaches 30

In [9]:
# Get country lelbel totals
df2 = df.groupby(["Country/Region", "Date"]).sum()

df2 = df2.loc[df2.Confirmed >= 30]
df2["log_Confirmed"] = np.log(df2["Confirmed"])
df2["log_Deaths"] = np.log(df2["Deaths"])
df2["Rate"] = df2["Deaths"] / df2["Confirmed"] 
df2 = df2.replace([np.inf, -np.inf], 0)
df2 = df2.reset_index()
df2["Date_ini"] = df2["Date"]
df2["Date"] = df2.groupby(["Country/Region"]).cumcount()
df2 = (df2
       .set_index(["Country/Region", "Date", "Date_ini"])
       .loc[:, ["Confirmed", "log_Confirmed", "Deaths", "log_Deaths", "rate"]]
      )

# Recalculate the rate
df2["rate"] = df2.Deaths / df2.Confirmed

# df2.loc[["France", "China"]]
# df2.loc["Turkey"]

C:\Users\mmenache\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


## Export the dashboard with pivottablejs.puivot_ii

In [10]:
# Export the interactive table / chart
pivot_ui(df2,outfile_path="country_comparison_rescaled.html")